In [1]:
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from datasets import load_dataset, DatasetDict, Dataset
from transformers import BertForSequenceClassification

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

import pandas as pd
import numpy as np
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras import Model, Input
from keras.callbacks import EarlyStopping,ModelCheckpoint
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
warnings.filterwarnings("ignore")
from accelerate import Accelerator
import swifter
from torch.utils.data.dataloader import DataLoader

/opt/anaconda3/envs/hugging-face-venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'swifter'

In [3]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir='fast',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=False
)

In [21]:
# train_dataloader = DataLoader(small_train_dataset, batch_size=training_args.per_device_train_batch_size)

# if training_args.gradient_checkpointing:
#     model.gradient_checkpointing_enable()

# accelerator = Accelerator()
# model, optimizer, dataloader = accelerator.prepare(model, train_dataloader)

# model.train()
# for step, batch in enumerate(dataloader, start=1):
#     loss = model(**batch).loss
#     loss = loss / training_args.gradient_accumulation_steps
#     accelerator.backward(loss)
#     if step % training_args.gradient_accumulation_steps == 0:
#         optimizer.step()
#         optimizer.zero_grad()

In [84]:
from transformers import pipeline, AutoModel, AutoTokenizer


In [103]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline

model = AutoModelForSequenceClassification.from_pretrained('NEW_MODEL')
tokenizer = AutoTokenizer.from_pretrained('NEW_TOKEN')

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

# Now you can use the pipeline
result = pipe("Your text here")
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[[{'label': 'LABEL_0', 'score': 0.4726463258266449}, {'label': 'LABEL_1', 'score': 0.5273536443710327}]]


In [106]:
def get_pred(text):
    pred = pipe(text)
    print(pred)
    return pred[0][1]['score']

In [107]:
final_df_path = 'test_essays.csv'
test_df = pd.read_csv(final_df_path)
test_df['generated'] = test_df['text'].apply(get_pred)
test_df = test_df[["id", "generated"]]
test_df

[[{'label': 'LABEL_0', 'score': 0.6134910583496094}, {'label': 'LABEL_1', 'score': 0.386508971452713}]]
[[{'label': 'LABEL_0', 'score': 0.6508035659790039}, {'label': 'LABEL_1', 'score': 0.3491964042186737}]]
[[{'label': 'LABEL_0', 'score': 0.6304652690887451}, {'label': 'LABEL_1', 'score': 0.36953476071357727}]]


,id,generated
0,0000aaaa,0.386509
1,1111bbbb,0.349196
2,2222cccc,0.369535


In [100]:
pipe